# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [4]:
import os
import sys

sys.path.append('..')
from utils import *
# Set your preferred name
USER = "Arthur"
# Set the data and output directories
DATA_ROOT = 'C:/Users/arthu/Downloads'
OUTPUT_ROOT = 'C:/Users/arthu/OneDrive/Documents/GitHub/output2'
TRAIT = 'Language-Abilities'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

utils.py has been loaded


In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [2]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

['TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'CrawlData.ipynb',
 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Endometrioid_Cancer_(UCEC)',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Glioblastoma_(GBM)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Kidney_Clear_Cell_Carcinoma_(KIRC)',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Rectal_Cancer_(READ)',
 'TCGA_Head_and_Neck_Cancer_(HNSC)',
 'TCGA_Large_Bcell_Lymphoma_(DLBC)',
 'TCGA_Uterine_Carcinosarcoma_(UCS)',
 'TCGA_Colon_Cancer_(COAD)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Bladder_Cancer_(BLCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Cervical_Ca

If no match is found, jump directly to GEO in Part 2.2

In [27]:
trait_subdir = "TCGA_Lung_Cancer_(LUNG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_LUNG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_HiSeqV2_PANCAN.gz')

In [28]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['ABSOLUTE_Ploidy',
 'ABSOLUTE_Purity',
 'AKT1',
 'ALK_translocation',
 'BRAF',
 'CBL',
 'CTNNB1',
 'Canonical_mut_in_KRAS_EGFR_ALK',
 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2',
 'EGFR']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0],
 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, nan, '121', nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['0', '1523', nan, '607', '426'],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  nan,
  nan,
  334.0,
  183.0],
 'stopped_smoking_year': [nan, nan, nan, nan, 1987.0],
 'year_of_initial_pathologic_diagnosis': [2009.0,
  2007.0,
  2007.0,
  2009.0,
  2009.0],
 'year_of_tobacco_smoking_onset': [nan, nan, nan, nan, 1963.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lung-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.639586,-0.195527,-0.324695,0.193167,-0.572812
TCGA-05-4249-01,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.997486,-0.276027,-1.219395,1.850967,-0.471212
TCGA-05-4250-01,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.959514,0.714373,0.112305,-0.532833,-0.003812
TCGA-05-4382-01,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,1.013722,-1.587117,0.359267,2.59051,0.806623,0.812114,0.367373,-0.494595,-1.062233,0.108588
TCGA-05-4384-01,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,0.110214,1.469373,0.194605,1.404667,-0.290712


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1100 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lung-Cancer', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Lung-Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Yuxuan/Lung-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [5]:
dataset = 'GEO'
trait_subdir = "Language-Abilities"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE103889',
 'GSE104554',
 'GSE109857',
 'GSE130786',
 'GSE130787',
 'GSE135079',
 'GSE150707',
 'GSE78278',
 'GSE84046',
 'GSE92640',
 'GSE99573']

Repeat the below steps for all the accession numbers

In [72]:
cohort = accession_num = "GSE99573"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('C:/Users/arthu/Downloads\\GEO\\Language-Abilities\\GSE99573\\GSE99573_family.soft.gz',
 'C:/Users/arthu/Downloads\\GEO\\Language-Abilities\\GSE99573\\GSE99573_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [73]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Clinical Performance of a Stool RNA Assay for Early Detection of Precancerous Adenomas and Colorectal Cancer"
!Series_summary	"Background and Aims: RNA biomarkers derived from sloughed enterocytes would provide an ideal, non-invasive method for early detection of colorectal cancer (CRC) and precancerous adenomas. To realize this goal, a highly reliable method to isolate preserved human RNA from stool samples is needed. Here we develop a protocol to identify RNA biomarkers associated with CRC to assess the use of these biomarkers for noninvasive screening of disease."
!Series_summary	"Methods: Stool samples were collected from 454 patients prior to a colonoscopy. A nucleic acid extraction protocol was developed to isolate human RNA from 330 stool samples and transcript abundances were estimated by microarray analysis. This 330-patient cohort was split into a training set of 265 individuals to develop a machine learning model and a testing set of 65 individuals to determin

In [74]:
clinical_data.head()

,!Sample_geo_accession,GSM2646622,GSM2646623,GSM2646624,GSM2646625,GSM2646626,GSM2646627,GSM2646628,GSM2646629,GSM2646630,...,GSM2646950,GSM2646951,GSM2646952,GSM2646953,GSM2646954,GSM2646955,GSM2646956,GSM2646957,GSM2646958,GSM2646959
0,!Sample_characteristics_ch1,tissue: Stool,tissue: Stool,tissue: Stool,tissue: Stool,tissue: Stool,tissue: Stool,tissue: Stool,tissue: Stool,tissue: Stool,...,tissue: Stool,tissue: Stool,tissue: Stool,tissue: Stool,tissue: Stool,tissue: Stool,tissue: Stool,tissue: Stool,tissue: Stool,tissue: Stool
1,!Sample_characteristics_ch1,disease status: Normal,disease status: Normal,disease status: Normal,disease status: Normal,disease status: Normal,disease status: Adenoma,disease status: Stage I,disease status: Stage III,disease status: Stage III,...,disease status: Stage IV,disease status: Stage I,disease status: Benign,disease status: Benign,disease status: Benign,disease status: Benign,disease status: Benign,disease status: Benign,disease status: Benign,disease status: Benign
2,!Sample_characteristics_ch1,Sex: Female,Sex: Male,Sex: Female,Sex: Female,Sex: Male,Sex: Male,Sex: Female,Sex: Male,Sex: Male,...,Sex: Male,Sex: Male,Sex: Female,Sex: Female,Sex: Male,Sex: Female,Sex: Male,Sex: Male,Sex: Female,Sex: Male
3,!Sample_characteristics_ch1,age (yr): 62,age (yr): 41,age (yr): 49,age (yr): 49,age (yr): 50,age (yr): 54,age (yr): 53,age (yr): 47,age (yr): 47,...,age (yr): 70,age (yr): 61,age (yr): 50,age (yr): 55,age (yr): 53,age (yr): 57,age (yr): 52,age (yr): 62,age (yr): 51,age (yr): 62
4,!Sample_characteristics_ch1,ethnicity: Caucasian/white,ethnicity: Caucasian/white,ethnicity: Caucasian/white,ethnicity: Caucasian/white,ethnicity: Caucasian/white,ethnicity: Caucasian/white,ethnicity: Caucasian/white,ethnicity: Caucasian/white,ethnicity: Caucasian/white,...,ethnicity: Caucasian/white,ethnicity: Caucasian/white,ethnicity: Caucasian/white,ethnicity: Caucasian/white,ethnicity: Caucasian/white,ethnicity: Caucasian/white,ethnicity: Caucasian/white,ethnicity: Caucasian/white,ethnicity: Caucasian/white,ethnicity: Caucasian/white


In [75]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['tissue: Stool'],
 1: ['disease status: Normal',
  'disease status: Adenoma',
  'disease status: Stage I',
  'disease status: Stage III',
  'disease status: Stage IV',
  'disease status: Stage II',
  'disease status: Benign'],
 2: ['Sex: Female', 'Sex: Male'],
 3: ['age (yr): 62',
  'age (yr): 41',
  'age (yr): 49',
  'age (yr): 50',
  'age (yr): 54',
  'age (yr): 53',
  'age (yr): 47',
  'age (yr): 48',
  'age (yr): 55',
  'age (yr): 63',
  'age (yr): 58',
  'age (yr): 60',
  'age (yr): 52',
  'age (yr): 56',
  'age (yr): 42',
  'age (yr): 68',
  'age (yr): 37',
  'age (yr): 44',
  'age (yr): 61',
  'age (yr): 67',
  'age (yr): 57',
  'age (yr): 64',
  'age (yr): 65',
  'age (yr): 35',
  'age (yr): 51',
  'age (yr): 71',
  'age (yr): 72',
  'age (yr): 70',
  'age (yr): 66',
  'age (yr): 69'],
 4: ['ethnicity: Caucasian/white'],
 5: ['smoking status: Yes', 'smoking status: No'],
 6: ['family history: No', 'family history: Yes'],
 7: ['set: Testing', 'set: Training', 'set: Not Used

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [76]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Language-Abilities\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Language-Abilities\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. Th

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [77]:
is_gene_availabe = False
trait_row = None
age_row = 3
gender_row = 2

trait_type = None

In [78]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

This cohort is not usable. Please skip the following steps and jump to the next accession number.


In [11]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'disease status: Normal':
        return 0  # Epilepsy present
    else:
        return 1  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age (yr): '): 
        try:
            return int(age_string.split('age (yr): ')[1].rstrip(''))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'Sex: Female':
        return 0
    elif gender_string == 'Sex: Male':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'tissue: Hippocampus'
example_age = 'age: 23y'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(1, None, None)

In [12]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)


,GSM2646622,GSM2646623,GSM2646624,GSM2646625,GSM2646626,GSM2646627,GSM2646628,GSM2646629,GSM2646630,GSM2646631,...,GSM2646950,GSM2646951,GSM2646952,GSM2646953,GSM2646954,GSM2646955,GSM2646956,GSM2646957,GSM2646958,GSM2646959
Language-Abilities,0,0,0,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Age,62,41,49,49,50,54,53,47,47,49,...,70,61,50,55,53,57,52,62,51,62
Gender,0,1,0,0,1,1,0,1,1,0,...,1,1,0,0,1,0,1,1,0,1


### Genetic data preprocessing and final filtering

In [13]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM2646622,GSM2646623,GSM2646624,GSM2646625,GSM2646626,GSM2646627,GSM2646628,GSM2646629,GSM2646630,GSM2646631,...,GSM2646950,GSM2646951,GSM2646952,GSM2646953,GSM2646954,GSM2646955,GSM2646956,GSM2646957,GSM2646958,GSM2646959
ID,,,,,,,,,,,,,,,,,,,,,
2824546_st,4.67947,3.69914,5.37322,4.05504,4.37088,4.03619,6.44081,4.91678,4.81161,6.31973,...,6.96657,5.11838,4.00952,4.42607,5.74545,3.42779,5.35258,4.44563,2.92628,5.51893
2824549_st,8.60108,5.50548,4.51721,6.07598,5.37494,6.51398,5.36177,5.86789,8.53433,7.34615,...,10.62002,7.82392,6.42365,8.37554,7.59447,8.05537,11.00541,6.68265,6.92273,8.82158
2824551_st,9.13175,5.15711,9.00256,10.09978,12.31998,7.13228,14.87278,7.88318,10.43565,11.22607,...,16.93644,13.52492,9.77677,13.28166,10.25665,12.13240,19.93150,9.94688,7.29800,10.13054
2824554_st,10.89461,8.30842,9.31762,12.97561,17.16499,7.30922,16.68641,7.94130,12.92613,13.19816,...,18.92546,15.02883,9.22639,14.18968,10.41925,13.01683,19.93150,10.77463,9.88145,15.11741
2827992_st,3.69238,7.23577,5.10228,4.03017,3.37050,4.44544,5.20728,4.66811,6.24677,8.83682,...,6.41920,7.92525,5.27315,5.58827,6.23887,4.25709,4.82687,3.65557,3.97461,4.61443


In [14]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['2824546_st',
 '2824549_st',
 '2824551_st',
 '2824554_st',
 '2827992_st',
 '2827995_st',
 '2827996_st',
 '2828010_st',
 '2828012_st',
 '2835442_st',
 '2835447_st',
 '2835453_st',
 '2835456_st',
 '2835459_st',
 '2835461_st',
 '2839509_st',
 '2839511_st',
 '2839513_st',
 '2839515_st',
 '2839517_st']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [15]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st', '2827995_st', '2827996_st', '2828010_st', '2828012_st', '2835442_st', '2835447_st', '2835453_st', '2835456_st', '2835459_st', '2835461_st', '2839509_st', '2839511_st', '2839513_st', '2839515_st', '2839517_st']\n"

If not required, jump directly to the gene normalization step

In [16]:
requires_gene_mapping = True

In [17]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST00000408384 // MIR1302-9 // microRN

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [18]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287

In [19]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'gene_assignment'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [20]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

998 input query terms found no hit:	['locus=scaffold1174:29274:29561:+ gene:ENSG00000179066', 'locus=scaffold2987:101738:101991:+ gene:E
1000 input query terms found no hit:	['100287225', '100287225', '100287226', '100287226', '100287227', '100287238', '100287284', '1002872
1000 input query terms found no hit:	['100505918', '100505920', '100505921', '100505929', '100505933', '100505938', '100505940', '1005059
1000 input query terms found no hit:	['100616486', '100616487', '100616489', '100616490', '100616491', '100616492', '100616493', '1006164
1000 input query terms found no hit:	['10438', '10439', '1044', '10440', '10443', '10444', '10445', '10445', '10446', '10447', '10449', '
1000 input query terms found no hit:	['11260', '112609', '11261', '112611', '112616', '11262', '11264', '11266', '11267', '11269', '11270
1000 input query terms found no hit:	['124871', '124872', '124912', '124923', '124925', '124925', '124930', '124935', '124936', '124936',
1000 input query terms found no hit

In [21]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [22]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 338 samples.


In [23]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Language-Abilities', the least common label is '0.0' with 111 occurrences. This represents 32.84% of the dataset.
The distribution of the feature 'Language-Abilities' in this dataset is fine.

Quartiles for 'Age':
  25%: 51.0
  50% (Median): 56.0
  75%: 62.0
Min: 34.0
Max: 75.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 139 occurrences. This represents 41.12% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [24]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

A new JSON file was created at: ../../output1/Yuxuan/Language-Abilities/cohort_info.json


In [25]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [26]:
from utils import *

In [27]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE99573'

In [28]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE99573,True,True,False,True,True,338,


In [29]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [30]:
merged_data.head()

,Language-Abilities,Age,Gender,BET1L,BOLA2,BOLA2B,CA14,CD79A,CLK3,EOS,...,SPRR2C,SPRR2D,SPRR2F,TBP,TPR,U1,GLA,HASPIN,SP1,TESMIN
0,0.0,62.0,0.0,5.58383,5.09692,5.09692,4.82381,5.295227,4.81541,5.02464,...,6.11522,6.11522,6.11522,5.410074,5.955096,6.05081,5.32946,6.35600,5.349367,5.08896
1,0.0,41.0,1.0,5.31565,5.30138,5.30138,4.65290,5.329220,4.82089,4.97764,...,5.59662,5.59662,5.59662,5.371055,6.137255,6.12838,5.21290,5.27611,5.360984,5.17372
2,0.0,49.0,0.0,5.49986,5.38284,5.38284,4.84919,5.440330,4.86002,4.99542,...,5.35165,5.35165,5.35165,5.331143,5.621839,6.12909,5.19776,6.76297,5.312664,4.92317
3,0.0,49.0,0.0,5.39709,5.38811,5.38811,5.10221,5.214910,5.27595,4.91909,...,5.62214,5.62214,5.62214,5.393022,5.694567,6.07534,5.34592,5.54120,5.326339,5.10696
4,0.0,50.0,1.0,5.39444,5.34461,5.34461,4.59554,5.324350,5.05524,5.00181,...,5.97287,5.97287,5.97287,5.444447,5.815639,6.16211,5.50025,6.58409,5.529115,4.95026


In [31]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [32]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [33]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [34]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [35]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 48.96% ± 5.54%


In [36]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 57.31% ± 8.10%


In [37]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 5. Discussion and report

In [38]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0653
Found 22 genes with non-zero coefficients associated with the trait 'Language-Abilities' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
 Variable   Coefficient  Absolute Coefficient
     FUSE  8.973221e-01          8.973221e-01
     CA14 -8.884510e-01          8.884510e-01
    BET1L -6.855938e-01          6.855938e-01
      SP1 -6.611190e-01          6.611190e-01
     NFYA  5.575705e-01          5.575705e-01
    PTPRF -4.721789e-01          4.721789e-01
   TESMIN -4.297876e-01          4.297876e-01
    GRID2  4.225442e-01          4.225442e-01
      GLA  4.225075e-01          4.225075e-01
   HASPIN  3.876394e-01          3.876394e-01
      TBP -2.399097e-01          2.399097e-01
     PARN  2.085300e-01          2.085300e-01
IGKV1D-12 -1.811528e-01          1.811528e-01
    CD79A  1.646982e-01          1.646982e-01
      FAS  1.459928e-01          

/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [39]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0653
Found 0 significant genes associated with the trait 'Language-Abilities' conditional on the factor 'Age', with corrected p-value < 0.05:
Empty DataFrame
Columns: [Variable, Coefficient, p_value, corrected_p_value]
Index: []


/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
